In [37]:
import numpy as np
import matplotlib.pyplot as plt

import time
from copy import deepcopy

import cv2

from IPython.display import clear_output

In [32]:
from client_mk1.fast_image import Image
from client_mk1.color import Color
from client_mk1.feat_2d import Circle2D
from client_mk1.searchers import do_linear_search, do_binary_search

In [33]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# Useful resources
[OpenCV tutorial](http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_video_display/py_video_display.html#playing-video-from-file)

# Define types for features

In [34]:
green = Color((30, 60, 100), (60, 255, 255))
orange = Color((5, 150, 130), (30, 255, 255))

greenCircle = Circle2D((1920/2, 1080/2), 400, green)
orangeCircle = Circle2D((1920/2, 1080/2), 400, orange)

greenSmall = Circle2D((1100, 700), 130, green)
orangeSmall = Circle2D((600, 700), 130, orange)

## Tests

# Capture and handle video

In [35]:
FILE = '../test_data/smooth_1.mp4'
cv2.namedWindow('raw_frame')

In [36]:
cap = cv2.VideoCapture(FILE)

def handleFrame(frame, features, verbose=False):
    image = Image(frame)

    for f in features:
        f.refine(image, verbose=verbose)
    
    return image


def main():
    features = deepcopy([greenSmall, orangeSmall])
    i=0
    while cap.isOpened():
        verbose = i%50 == 0
        if verbose:
            clear_output()
        i += 1
        
        startTime = time.time()
        ret, frame = cap.read()
        readTime = time.time()

        if frame is None or len(frame) <= 0:
            print "End of video file reached"
            break

        # Do work here
        image = handleFrame(frame, features, verbose=verbose)
        
        finishTime = time.time()

        for f in features:
            f.draw(frame)
            
        image.draw(frame)
        scaled = cv2.resize(frame, dsize=None, fx=0.5, fy=0.5)
        cv2.imshow('raw_frame', scaled)
        
        drawnTime = time.time()

        cv2.waitKey(1)
        
        if verbose:
            processingTime = (finishTime-readTime)
            print "{0:.1f}ms reading and drawing image, {1:.1f}ms ({3}fps) processing, {2} pixels accessed\n".format(
                (readTime-startTime + drawnTime-finishTime)*1000, 
                processingTime*1000, 
                len(image.pixelsAccessed),
                int(1/processingTime))
        
main()

10.1ms reading and drawing image, 2.1ms (474fps) processing, 74 pixels accessed

End of video file reached


# Profile things

In [15]:
cap = cv2.VideoCapture(FILE)

def bench():
    features = deepcopy([greenSmall])
    for _ in range(60):
        ret, frame = cap.read()
        handleFrame(frame, features)
    
%lprun -r -f doBinarySearch bench()